In [1]:
import lal, lalsimulation
import json
import numpy as np
import sys
sys.path.append('../src')
sys.path.append('../scripts')
import standard_gwtransf as gw
import os
import pycbc
import pycbc.noise
import pycbc.psd
from pycbc.frame import query_and_read_frame
from pycbc.types import timeseries
from pycbc.psd import inverse_spectrum_truncation, interpolate
from pycbc.waveform import get_fd_waveform, get_td_waveform
import pycbc.filter.matchedfilter as mfilter
from pycbc import frame as Fr
import scipy.signal
import imrtgrutils_final as tgr
import nr_fits as nr
import glob
import matplotlib.pyplot as plt
import lalsimulation
import h5py

# Noise

### Read in science quality data stretches from O2, coincident over all 3 detectors: H1,L1,V1, and select the ones greater that 512 seconds.
Also choose geocentric times as the centres of these segments.

In [2]:
o2_L1H1V1_start_times, o2_L1H1V1_end_times, o2_L1H1V1_stretches = np.loadtxt('../data/systematics_error_characterisation/o2_L1H1V1_quality_segments.txt', unpack=True)

print 'total stretches of coincident data:', len(o2_L1H1V1_stretches)

idx, = [np.where(o2_L1H1V1_stretches>512)]
print 'total stretches of coincident data greater than 512 seconds:', len(o2_L1H1V1_stretches[idx])

o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_stretches = o2_L1H1V1_start_times[idx], o2_L1H1V1_end_times[idx], o2_L1H1V1_stretches[idx]
o2_L1H1V1_512_geocentric_times = o2_L1H1V1_start_times[idx] + (o2_L1H1V1_start_times[idx] - o2_L1H1V1_end_times[idx])/2.

#np.savetxt('./o2_L1H1V1_512_times.dat', np.c_[o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_stretches, o2_L1H1V1_512_geocentric_times], header="o2_L1H1V1_512_start_times o2_512_L1H1V1_end_times o2_L1H1V1_512_stretches o2_L1H1V1_512_geocentric_times")

total stretches of coincident data: 284
total stretches of coincident data greater than 512 seconds: 187


# Signal

### Read in details of the LVC-NR catalog, and separate out the waveforms from the SXS catalog

In [3]:
f_lvcnr = open('/home/abhirup/ligo-nr-data/lvcnr-lfs.json', 'r')
lvcnr_data = json.load(f_lvcnr)
f_lvcnr.close()

sxs_idx_list = []

for idx in range(len(lvcnr_data)):
    if lvcnr_data[idx]['NR-group'] == 'SXS':
        sxs_idx_list.append(idx)

### Specifying the parameters of the binary to be simulated using a specific SXS-NR waveform
The SXS waveform has information about the mass ratio, q or ((m1,m2) normalised to a total mass M = 1 M_sun) and the spins: (s1x, s1y, s1z, s2x, s2y, s2z). We choose the other parameters (M, iota, pol, ra, dec, snr) by uniformly sampling between reasonable ranges.

In [4]:
N_inj = 1000
f_lower = 25.
f_ref = f_lower
deltaT, phiRef = 1./4096., 0.0

#signal_params_file = open('./signal_params_list.dat', 'w')
#signal_params_file.write("#sxs_idx path m1 m2 M s1x s1y s1z s2x s2y s2z snr iota pol ra dec Mf af f_isco_Kerr\n")

for N_inj_idx in range(N_inj):
    try: 
        sxs_idx = np.random.choice(sxs_idx_list)
        path = '/home/abhirup/ligo-nr-data/lvcnr-lfs/'+lvcnr_data[sxs_idx]['path']
        
        M = np.random.uniform(20,160) # total mass chosen random from distribution uniform in [40,160] M_sun
        snr = np.random.uniform(10,50)   # choose a random SNR uniformly distributed in [15,40]
        iota = np.arccos(np.random.uniform(-1., 1.))
        pol = np.random.uniform(0., np.pi)
        ra = np.random.uniform(0., 2.*np.pi)
        dec = np.random.uniform(-np.pi/2., np.pi/2.)
        
        m1, m2 = lvcnr_data[sxs_idx]['mass1']*M, lvcnr_data[sxs_idx]['mass2']*M
        s1x,s1y,s1z,s2x,s2y,s2z = lalsimulation.SimInspiralNRWaveformGetSpinsFromHDF5File(f_ref, M, (path).encode('utf-8'))
        

        # computing f_isco
        s1 = np.sqrt(s1x**2. + s1y**2. + s1z**2.)
        s2 = np.sqrt(s2x**2. + s2y**2. + s2z**2.)
        tilt1 = np.arccos(s1z/s1)
        tilt2 = np.arccos(s2z/s2)
        phi12 = np.arctan(s2y/s2x) - np.arctan(s1y/s1x)

        # calculate the mass and spin of the final BH 
        if lvcnr_data[sxs_idx]['simulation-type'] == 'non-spinning':
            Mf, af = tgr.calc_final_mass_spin(m1, m2, s1, s2, s1z, s2z, tilt1, tilt2, phi12, 'nospin_Pan2011')
        elif lvcnr_data[sxs_idx]['simulation-type'] == 'aligned-spins':
            Mf, af = tgr.calc_final_mass_spin(m1, m2, s1, s2, s1z, s2z, tilt1, tilt2, phi12, 'nonprecspin_Healy2014')
        elif lvcnr_data[sxs_idx]['simulation-type'] == 'precessing':
            Mf, af = tgr.calc_final_mass_spin(m1, m2, s1, s2, s1z, s2z, tilt1, tilt2, phi12, 'bbh_average_fits_precessing')

        f_isco_Kerr = nr.calc_isco_freq(af)/(Mf*lal.MTSUN_SI)
        
        #signal_params_file.write("%d %s %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f %f\n"%(sxs_idx, lvcnr_data[sxs_idx]['path'], m1, m2, M, s1x, s1y, s1z, s2x, s2y, s2z, snr, iota, pol, ra, dec, Mf, af, f_isco_Kerr))
        
    except RuntimeError:
        x = 1
        
#signal_params_file.close()

Truncating eta from above to 0.25. This should only be necessary in some rounding corner cases, but better check your m1 and m2 inputs...
Truncating eta from above to 0.25. This should only be necessary in some rounding corner cases, but better check your m1 and m2 inputs...
Truncating eta from above to 0.25. This should only be necessary in some rounding corner cases, but better check your m1 and m2 inputs...
Truncating eta from above to 0.25. This should only be necessary in some rounding corner cases, but better check your m1 and m2 inputs...
Truncating eta from above to 0.25. This should only be necessary in some rounding corner cases, but better check your m1 and m2 inputs...
Truncating eta from above to 0.25. This should only be necessary in some rounding corner cases, but better check your m1 and m2 inputs...


## Injections

In [2]:
signal_params_list = np.genfromtxt('./signal_params_list.dat', names=True, dtype=None)
o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_stretches, o2_L1H1V1_512_geocentric_times = np.loadtxt('./o2_L1H1V1_512_times.dat', unpack=True)

/home/abhirup/src/pycbc_new/lib/python2.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


### IMRPPv2: HLV Design PSD

--asd-file H1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1200087-v18-aLIGO_DESIGN.txt \

L1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1200087-v18-aLIGO_DESIGN.txt \

V1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1200087-v18-AdV_DESIGN.txt

In [12]:
signal_params_list = np.genfromtxt('./signal_params_list.dat', names=True, dtype=None)
o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_stretches, o2_L1H1V1_512_geocentric_times = np.loadtxt('./o2_L1H1V1_512_times.dat', unpack=True)
out_dir = "/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/imrppv2_injection_psdmodel_zeronoise"
                                
# Pick first 50 injections

example_config = '/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/imrppv2_injection_psdmodel_zeronoise/imrppv2_injection_psdmodel_zeronoise_example_config.ini'
sub_dir_list = ['IMR', 'inspiral', 'post-inspiral']
                                
for (signal_params, gps_start_time, gps_end_time, geocentric_time) in zip(signal_params_list[:50], o2_L1H1V1_512_start_times[:50], o2_512_L1H1V1_end_times[:50], o2_L1H1V1_512_geocentric_times[:50]):


    sxs_idx, path, m1, m2, M, s1x, s1y, s1z, s2x, s2y, s2z, snr, iota, pol, ra, dec, Mf, af, f_isco_Kerr = signal_params
    
    command = "pycbc_generate_hwinj --approximant IMRPhenomPv2 --order pseudoFourPN \
          --mass1 %f --mass2 %f \
          --spin1x %f --spin1y %f --spin1z %f \
          --spin2x %f --spin2y %f --spin2z %f \
          --ra %f --dec %f --polarization %f --inclination %f \
          --network-snr %f \
          --geocentric-end-time %d --gps-start-time %d --gps-end-time %d \
          --instruments H1 L1 V1 \
          --asd-file H1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1200087-v18-aLIGO_DESIGN.txt \
          L1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1200087-v18-aLIGO_DESIGN.txt \
          V1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1200087-v18-AdV_DESIGN.txt \
          --taper TAPER_STARTEND --waveform-low-frequency-cutoff 15.0 --low-frequency-cutoff 25.0 \
          --sample-rate 4096"%(m1, m2, s1x, s1y, s1z, s2x, s2y, s2z, ra, dec, pol, iota, snr, geocentric_time, gps_start_time, gps_end_time)

    os.system(command)

    for sub_dir in sub_dir_list:
                                
        os.system('mkdir -p %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system('cp %s %s/%d/%s/config.ini'%(example_config, out_dir, geocentric_time, sub_dir))
        os.system('cp hwinjcbc* %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system("sed -i 's/imrppv2_injection_psdmodel_zeronoise/imrppv2_injection_psdmodel_zeronoise\/%d\/%s/g' %s/%d/%s/config.ini"%(geocentric_time, sub_dir, out_dir, geocentric_time, sub_dir))  
            
        if sub_dir == 'inspiral':
            os.system("""sed -i "/^flow/a fhigh=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
        elif sub_dir == 'post-inspiral':
            os.system("""sed -i "s/^flow.*/flow=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}/g" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
    
    os.system('rm -r hwinjcbc*')


/home/abhirup/src/pycbc_new/lib/python2.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


### IMRPPv2: O2 noise [HL: unclean C02; V:V1O2Repro2A]

--frame-type H1:H1_HOFT_C02 L1:L1_HOFT_C02 V1:V1O2Repro2A \
          --channel-name H1:DCS-CALIB_STRAIN_C02 L1:DCS-CALIB_STRAIN_C02 V1:Hrec_hoft_V1O2Repro2A_16384Hz \
          --psd-estimation median --psd-estimation median --psd-segment-length 16 --psd-segment-stride 8 \
          --pad-data 8 --strain-high-pass 1\

In [ ]:
out_dir = "/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/imrppv2_injection_o1o2_noise"
                                
# Pick first 50 injections

example_config = '/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/imrppv2_injection_o1o2_noise/imrppv2_injection_o1o2_noise_example_config.ini'
sub_dir_list = ['IMR', 'inspiral', 'post-inspiral']

# Pick first 50 injections

for (signal_params, gps_start_time, gps_end_time, geocentric_time) in zip(signal_params_list[:1], o2_L1H1V1_512_start_times[:1], o2_512_L1H1V1_end_times[:1], o2_L1H1V1_512_geocentric_times[:1]):


    sxs_idx, path, m1, m2, M, s1x, s1y, s1z, s2x, s2y, s2z, snr, iota, pol, ra, dec, Mf, af, f_isco_Kerr = signal_params
    
    command = "pycbc_generate_hwinj --approximant IMRPhenomPv2 --order pseudoFourPN \
          --mass1 %f --mass2 %f \
          --spin1x %f --spin1y %f --spin1z %f \
          --spin2x %f --spin2y %f --spin2z %f \
          --ra %f --dec %f --polarization %f --inclination %f \
          --network-snr %f \
          --geocentric-end-time %d --gps-start-time %d --gps-end-time %d \
          --instruments H1 L1 V1 \
          --frame-type H1:H1_HOFT_C02 L1:L1_HOFT_C02 V1:V1O2Repro2A \
          --channel-name H1:DCS-CALIB_STRAIN_C02 L1:DCS-CALIB_STRAIN_C02 V1:Hrec_hoft_V1O2Repro2A_16384Hz \
          --psd-estimation median --psd-estimation median --psd-segment-length 16 --psd-segment-stride 8 \
          --pad-data 8 --strain-high-pass 1\
          --taper TAPER_STARTEND --waveform-low-frequency-cutoff 15.0 --low-frequency-cutoff 25.0 \
          --sample-rate 4096"%(m1, m2, s1x, s1y, s1z, s2x, s2y, s2z, ra, dec, pol, iota, snr, geocentric_time, gps_start_time, gps_end_time)

    os.system(command)

    for sub_dir in sub_dir_list:
                                
        os.system('mkdir -p %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system('cp %s %s/%d/%s/config.ini'%(example_config, out_dir, geocentric_time, sub_dir))
        os.system('cp hwinjcbc* %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system("sed -i 's/imrppv2_injection_o1o2_noise/imrppv2_injection_o1o2_noise\/%d\/%s/g' %s/%d/%s/config.ini"%(geocentric_time, sub_dir, out_dir, geocentric_time, sub_dir))  
            
        if sub_dir == 'inspiral':
            os.system("""sed -i "/^flow/a fhigh=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
        elif sub_dir == 'post-inspiral':
            os.system("""sed -i "s/^flow.*/flow=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}/g" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
    
    os.system('rm -r hwinjcbc*')

### IMRPPv2: CE PSD
Using one CE detector with PSD:LIGO-P1600143-v18-CE.txt placed at the location of H1
Waveform generation: 5 Hz
PSD computation: 10 Hz

--asd-file H1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1600143-v18-CE.txt

In [3]:
signal_params_list = np.genfromtxt('./signal_params_list.dat', names=True, dtype=None)
o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_stretches, o2_L1H1V1_512_geocentric_times = np.loadtxt('./o2_L1H1V1_512_times.dat', unpack=True)
out_dir = "/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/imrppv2_injection_psdmodel_CE_zeronoise"
                                
# Pick first 50 injections

example_config = '/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/imrppv2_injection_psdmodel_CE_zeronoise/imrppv2_injection_psdmodel_CE_zeronoise_example_config.ini'
sub_dir_list = ['IMR', 'inspiral', 'post-inspiral']
                                
for (signal_params, gps_start_time, gps_end_time, geocentric_time) in zip(signal_params_list, o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_geocentric_times):


    sxs_idx, path, m1, m2, M, s1x, s1y, s1z, s2x, s2y, s2z, snr, iota, pol, ra, dec, Mf, af, f_isco_Kerr = signal_params
    
    command = "pycbc_generate_hwinj --approximant IMRPhenomPv2 --order pseudoFourPN \
          --mass1 %f --mass2 %f \
          --spin1x %f --spin1y %f --spin1z %f \
          --spin2x %f --spin2y %f --spin2z %f \
          --ra %f --dec %f --polarization %f --inclination %f \
          --network-snr %f \
          --geocentric-end-time %d --gps-start-time %d --gps-end-time %d \
          --instruments H1 \
          --asd-file H1:/home/abhirup/opt/lalsuite_master_20190722_1feab6d9/share/lalsimulation/LIGO-P1600143-v18-CE.txt \
          --taper TAPER_STARTEND --waveform-low-frequency-cutoff 5.0 --low-frequency-cutoff 10.0 \
          --sample-rate 4096"%(m1, m2, s1x, s1y, s1z, s2x, s2y, s2z, ra, dec, pol, iota, snr, geocentric_time, gps_start_time, gps_end_time)

    os.system(command)

    for sub_dir in sub_dir_list:
                                
        os.system('mkdir -p %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system('cp %s %s/%d/%s/config.ini'%(example_config, out_dir, geocentric_time, sub_dir))
        os.system('cp hwinjcbc* %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system("sed -i 's/imrppv2_injection_psdmodel_CE_zeronoise/imrppv2_injection_psdmodel_CE_zeronoise\/%d\/%s/g' %s/%d/%s/config.ini"%(geocentric_time, sub_dir, out_dir, geocentric_time, sub_dir))  
            
        if sub_dir == 'inspiral':
            os.system("""sed -i "/^flow/a fhigh=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
        elif sub_dir == 'post-inspiral':
            os.system("""sed -i "s/^flow.*/flow=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}/g" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
    
    os.system('rm -r hwinjcbc*')


/home/abhirup/src/pycbc_new/lib/python2.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


### IMRPPv2: ET-CE PSD
Using one CE detector with PSD:LIGO-P1600143-v18-CE.txt placed at the location of H1 and one ET detector with PSD:LIGO-P1600143-v18-ET_D.txt placed at the location of V1
Waveform generation: 5 Hz
PSD computation: 10 Hz

--asd-file H1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1600143-v18-CE.txt \ 
    V1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1600143-v18-ET_D.txt

In [6]:
signal_params_list = np.genfromtxt('./signal_params_list.dat', names=True, dtype=None)
o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_stretches, o2_L1H1V1_512_geocentric_times = np.loadtxt('./o2_L1H1V1_512_times.dat', unpack=True)
out_dir = "/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/imrppv2_injection_psdmodel_CE-ET_zeronoise"
                                
# Pick first 50 injections

example_config = '/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/imrppv2_injection_psdmodel_CE-ET_zeronoise/imrppv2_injection_psdmodel_CE-ET_zeronoise_example_config.ini'
sub_dir_list = ['IMR', 'inspiral', 'post-inspiral']
                                
for (signal_params, gps_start_time, gps_end_time, geocentric_time) in zip(signal_params_list, o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_geocentric_times):


    sxs_idx, path, m1, m2, M, s1x, s1y, s1z, s2x, s2y, s2z, snr, iota, pol, ra, dec, Mf, af, f_isco_Kerr = signal_params
    
    command = "pycbc_generate_hwinj --approximant IMRPhenomPv2 --order pseudoFourPN \
          --mass1 %f --mass2 %f \
          --spin1x %f --spin1y %f --spin1z %f \
          --spin2x %f --spin2y %f --spin2z %f \
          --ra %f --dec %f --polarization %f --inclination %f \
          --network-snr %f \
          --geocentric-end-time %d --gps-start-time %d --gps-end-time %d \
          --instruments H1 V1\
          --asd-file H1:/home/abhirup/opt/lalsuite_master_20190722_1feab6d9/share/lalsimulation/LIGO-P1600143-v18-CE.txt \
          V1:/home/abhirup/opt/lalsuite_master_20190722_1feab6d9/share/lalsimulation/LIGO-P1600143-v18-ET_D.txt \
          --taper TAPER_STARTEND --waveform-low-frequency-cutoff 5.0 --low-frequency-cutoff 10.0 \
          --sample-rate 4096"%(m1, m2, s1x, s1y, s1z, s2x, s2y, s2z, ra, dec, pol, iota, snr, geocentric_time, gps_start_time, gps_end_time)

    os.system(command)

    for sub_dir in sub_dir_list:
                                
        os.system('mkdir -p %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system('cp %s %s/%d/%s/config.ini'%(example_config, out_dir, geocentric_time, sub_dir))
        os.system('cp hwinjcbc* %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system("sed -i 's/imrppv2_injection_psdmodel_CE-ET_zeronoise/imrppv2_injection_psdmodel_CE-ET_zeronoise\/%d\/%s/g' %s/%d/%s/config.ini"%(geocentric_time, sub_dir, out_dir, geocentric_time, sub_dir))  
            
        if sub_dir == 'inspiral':
            os.system("""sed -i "/^flow/a fhigh=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
        elif sub_dir == 'post-inspiral':
            os.system("""sed -i "s/^flow.*/flow=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}/g" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
    
    os.system('rm -r hwinjcbc*')

/home/abhirup/src/pycbc_new/lib/python2.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


### NR-SXS: HLV Design PSD
* used master branch of lalsuite
* replace sim_inspiral:process_id:0 by process:process_id:0 in sim_inspiral:table

In [3]:
signal_params_list = np.genfromtxt('./signal_params_list.dat', names=True, dtype=None)
o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_stretches, o2_L1H1V1_512_geocentric_times = np.loadtxt('./o2_L1H1V1_512_times.dat', unpack=True)
out_dir = "/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/sxs_injection_psdmodel_zeronoise"
                                
# Pick first 50 injections

example_config = '/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/sxs_injection_psdmodel_zeronoise/sxs_injection_psdmodel_zeronoise_example_config.ini'
sub_dir_list = ['IMR', 'inspiral', 'post-inspiral']
                                
for (signal_params, gps_start_time, gps_end_time, geocentric_time) in zip(signal_params_list[200:400], o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_geocentric_times):


    sxs_idx, path, m1, m2, M, s1x, s1y, s1z, s2x, s2y, s2z, snr, iota, pol, ra, dec, Mf, af, f_isco_Kerr = signal_params
    
    command = "pycbc_generate_hwinj --numrel-data /home/abhirup/ligo-nr-data/lvcnr-lfs/%s \
          --approximant NR_hdf5 --order pseudoFourPN \
          --mass1 %f --mass2 %f \
          --spin1x %f --spin1y %f --spin1z %f \
          --spin2x %f --spin2y %f --spin2z %f \
          --ra %f --dec %f --polarization %f --inclination %f \
          --network-snr %f \
          --geocentric-end-time %d --gps-start-time %d --gps-end-time %d \
          --instruments H1 L1 V1 \
          --asd-file H1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1200087-v18-aLIGO_DESIGN.txt \
          L1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1200087-v18-aLIGO_DESIGN.txt \
          V1:/home/abhirup/src/lalsuite/lalsimulation/src/LIGO-P1200087-v18-AdV_DESIGN.txt \
          --taper TAPER_STARTEND --waveform-low-frequency-cutoff 20.0 --low-frequency-cutoff 25.0 \
          --sample-rate 4096"%(path, m1, m2, s1x, s1y, s1z, s2x, s2y, s2z, ra, dec, pol, iota, snr, geocentric_time, gps_start_time, gps_end_time)

    os.system(command)
    os.system('gunzip *.xml.gz')
    os.system("sed -i 's/sim_inspiral:process_id:0/process:process_id:0/g' *.xml")
    
    for sub_dir in sub_dir_list:
                                
        os.system('mkdir -p %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system('cp %s %s/%d/%s/config.ini'%(example_config, out_dir, geocentric_time, sub_dir))
        os.system('cp hwinjcbc* %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system("sed -i 's/sxs_injection_psdmodel_zeronoise/sxs_injection_psdmodel_zeronoise\/%d\/%s/g' %s/%d/%s/config.ini"%(geocentric_time, sub_dir, out_dir, geocentric_time, sub_dir))  
            
        if sub_dir == 'inspiral':
            os.system("""sed -i "/^flow/a fhigh=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
        elif sub_dir == 'post-inspiral':
            os.system("""sed -i "s/^flow.*/flow=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}/g" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
    
    os.system('rm -r hwinjcbc*')

/home/abhirup/src/pycbc_new/lib/python2.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


### NR-SXS: O2 noise [HL: unclean C02; V:V1O2Repro2A]

* replace sim_inspiral:process_id:0 by process:process_id:0 in sim_inspiral:table
* used master branch of lalsuite
  * ignore-state-vector = True
  * datafind = /usr/bin/gw_data_find

--frame-type H1:H1_HOFT_C02 L1:L1_HOFT_C02 V1:V1O2Repro2A \
          --channel-name H1:DCS-CALIB_STRAIN_C02 L1:DCS-CALIB_STRAIN_C02 V1:Hrec_hoft_V1O2Repro2A_16384Hz \
          --psd-estimation median --psd-estimation median --psd-segment-length 16 --psd-segment-stride 8 \
          --pad-data 8 --strain-high-pass 1\

In [3]:
signal_params_list = np.genfromtxt('./signal_params_list.dat', names=True, dtype=None)
o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_stretches, o2_L1H1V1_512_geocentric_times = np.loadtxt('./o2_L1H1V1_512_times.dat', unpack=True)
out_dir = "/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/sxs_injection_o1o2_noise"
                                
# Pick first 50 injections

example_config = '/home/abhirup/Documents/Work/testGR_IR/runs/systematics_error_characterisation/new_runs/sxs_injection_o1o2_noise/sxs_injection_o1o2_noise_example_config.ini'
sub_dir_list = ['IMR', 'inspiral', 'post-inspiral']
                                
for (signal_params, gps_start_time, gps_end_time, geocentric_time) in zip(signal_params_list[200:400], o2_L1H1V1_512_start_times, o2_512_L1H1V1_end_times, o2_L1H1V1_512_geocentric_times):


    sxs_idx, path, m1, m2, M, s1x, s1y, s1z, s2x, s2y, s2z, snr, iota, pol, ra, dec, Mf, af, f_isco_Kerr = signal_params
    
    command = "pycbc_generate_hwinj --numrel-data /home/abhirup/ligo-nr-data/lvcnr-lfs/%s \
          --approximant NR_hdf5 --order pseudoFourPN \
          --mass1 %f --mass2 %f \
          --spin1x %f --spin1y %f --spin1z %f \
          --spin2x %f --spin2y %f --spin2z %f \
          --ra %f --dec %f --polarization %f --inclination %f \
          --network-snr %f \
          --geocentric-end-time %d --gps-start-time %d --gps-end-time %d \
          --instruments H1 L1 V1 \
          --frame-type H1:H1_HOFT_C02 L1:L1_HOFT_C02 V1:V1O2Repro2A \
          --channel-name H1:DCS-CALIB_STRAIN_C02 L1:DCS-CALIB_STRAIN_C02 V1:Hrec_hoft_V1O2Repro2A_16384Hz \
          --psd-estimation median --psd-estimation median --psd-segment-length 16 --psd-segment-stride 8 \
          --pad-data 8 --strain-high-pass 1 \
          --taper TAPER_STARTEND --waveform-low-frequency-cutoff 20.0 --low-frequency-cutoff 25.0 \
          --sample-rate 4096"%(path, m1, m2, s1x, s1y, s1z, s2x, s2y, s2z, ra, dec, pol, iota, snr, geocentric_time, gps_start_time, gps_end_time)

    os.system(command)
    os.system('gunzip *.xml.gz')
    os.system("sed -i 's/sim_inspiral:process_id:0/process:process_id:0/g' *.xml")
    
    for sub_dir in sub_dir_list:
                                
        os.system('mkdir -p %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system('cp %s %s/%d/%s/config.ini'%(example_config, out_dir, geocentric_time, sub_dir))
        os.system('cp hwinjcbc* %s/%d/%s'%(out_dir, geocentric_time, sub_dir))
        os.system("sed -i 's/sxs_injection_o1o2_noise/sxs_injection_o1o2_noise\/%d\/%s/g' %s/%d/%s/config.ini"%(geocentric_time, sub_dir, out_dir, geocentric_time, sub_dir))  
            
        if sub_dir == 'inspiral':
            os.system("""sed -i "/^flow/a fhigh=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
        elif sub_dir == 'post-inspiral':
            os.system("""sed -i "s/^flow.*/flow=\{'H1':%.2f,'L1':%.2f,'V1':%.2f\}/g" %s/%d/%s/config.ini"""%(f_isco_Kerr, f_isco_Kerr, f_isco_Kerr, out_dir, geocentric_time, sub_dir))
    
    os.system('rm -r hwinjcbc*')

/home/abhirup/src/pycbc_new/lib/python2.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  """Entry point for launching an IPython kernel.


In [9]:
signal_params_list = np.genfromtxt('./signal_params_list.dat', names=True, dtype=None)

bam BH BH precessing 0.247934057846 0.00369902
bam BH BH precessing 0.247934057846 0.00372695
bam BH BH precessing 0.247933396661 0.00560487
bam BH BH precessing 0.247933446254 0.00767789
bam BH BH precessing 0.247934289233 0.00425378
bam BH BH precessing 0.247934016529 0.00464311
bam BH BH precessing 0.247933446254 0.00372658
bam BH BH precessing 0.247933446254 0.00388168
bam BH BH precessing 0.247934181806 0.00639483
bam BH BH precessing 0.247933446254 0.00450201
bam BH BH precessing 0.247934140489 0.00362493
bam BH BH precessing 0.247934132223 0.00617173
bam BH BH precessing 0.247934132223 0.00633857
bam BH BH precessing 0.247933396661 0.00314537
bam BH BH precessing 0.247934016529 0.00247954
bam BH BH precessing 0.247933446254 0.00243468
bam BH BH precessing 0.247933446254 0.00293799
bam BH BH precessing 0.247934305767 0.00401866
bam BH BH precessing 0.247934305767 0.0039138
bam BH BH precessing 0.247934305767 0.00395377
bam BH BH precessing 0.247934057846 0.00399293
bam BH BH prec

SXS BH BH non-spinning 0.101115104382 0.0002243
SXS BH BH non-spinning 0.0849847476573 0.0004
SXS BH BH non-spinning 0.0849884430419 0.0002629
SXS BH BH non-spinning 0.129802788723 0.0004227
SXS BH BH non-spinning 0.129811746529 0.00022
SXS BH BH non-spinning 0.247886365018 0.0022495
SXS BH BH non-spinning 0.247886810998 0.0021797
SXS BH BH non-spinning 0.247886558722 0.0003368
SXS BH BH non-spinning 0.247887061615 0.0002996
SXS BH BH non-spinning 0.247887442649 0.0002044
SXS BH BH non-spinning 0.0922245265556 0.000278
SXS BH BH non-spinning 0.0922225553041 6.77e-05
SXS BH BH non-spinning 0.179270266792 0.0004982
SXS BH BH non-spinning 0.179264668722 0.0003554
SXS BH BH non-spinning 0.179273336156 0.0004137
SXS BH BH non-spinning 0.210619501811 9.17e-05
SXS BH BH non-spinning 0.210612434525 0.0001021
SXS BH BH non-spinning 0.210617549821 0.0001406
SXS BH BH aligned-spins 0.10937615321 7.73e-05
SXS BH BH aligned-spins 0.10937541812 0.0001278
SXS BH BH aligned-spins 0.109378548768 9.59e-

Georgia Tech BH BH non-spinning 0.21120294 0.00381976792376
Georgia Tech BH BH non-spinning 0.21852237 0.00381453039102
Georgia Tech BH BH non-spinning 0.24943311 0.00392164052753
Georgia Tech BH BH non-spinning 0.20761246 0.00382231355982
Georgia Tech BH BH non-spinning 0.19390582 0.00389834363127
Georgia Tech BH BH non-spinning 0.20061728 0.00382515162792
Georgia Tech BH BH non-spinning 0.19722425 0.00387112453759
Georgia Tech BH BH non-spinning 0.19066404 0.00388442847058
Georgia Tech BH BH non-spinning 0.1814059 0.00402787378513
Georgia Tech BH BH non-spinning 0.15089163 0.00503741887551
Georgia Tech BH BH non-spinning 0.18441404 0.00390626223322
Georgia Tech BH BH non-spinning 0.17847485 0.00403903287049
Georgia Tech BH BH non-spinning 0.14876033 0.00511655235059
Georgia Tech BH BH non-spinning 0.17561983 0.00413733896432
Georgia Tech BH BH non-spinning 0.17283951 0.0034104101664
Georgia Tech BH BH non-spinning 0.17013233 0.00354946367185
Georgia Tech BH BH non-spinning 0.16493056